<a href="https://colab.research.google.com/github/twoheartKES/weather-sales-forecast-ai/blob/main/notebooks/0_weather_disease(Preprocessing).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

AI 기상데이터 질병 예측 모델
데이터 전처리
1. 주간 질병확진자 데이터를 7로 단순 배분하여 일별 확지자로 데이터를 전처리한다.
2. 질병데이터의 일간 확진자데이터와 일별 기상기후에 대한 매핑을 위해 기상 관측소 마다 지역을 매핑되도록 구상한다.
3. 질병 확진 데이터 지역과 기상 관측소 데이터를 매핑한다.
4. feature dataset을 추출한다.


## 코드 개요 및 주요 기능

이 코드는 특정 지역별, 질병별 주간 데이터를 불러와 일주일(7일)로 균등하게 나눈 뒤, 실제 날짜와 매칭하여 일별 데이터셋을 생성합니다.

1. **연도별 반복 처리**: 2023년부터 2025년까지의 데이터를 루프를 통해 처리합니다.
2. **데이터 일 단위 분할**: 각 주차별 확진자 수를 7로 나누어 해당 주의 일별 확진자 수(`daily_cases`)를 계산합니다.
3. **날짜 매칭**: 해당 연도의 첫 번째 일요일을 기준으로 각 주차와 일자(`date`)를 생성합니다.
4. **데이터 필터링**: 설정된 `date_ranges` 기간에 해당하는 데이터만 추출하여 정제합니다.
5. **결과 저장**: 지역, 날짜, 질병명을 기준으로 정렬하여 UTF-8-SIG 인코딩 방식의 CSV 파일로 저장합니다.

```

[파일 시스템 구조]


/content/drive/MyDrive/
└── AI기상-질병데이터/
    ├── {year}년확진자/                    # (Input) 연도별 원본 폴더
    │   ├── kdca_{year}_week_경기_가평군_075.csv
    │   ├── kdca_{year}_week_경기_고양시_덕양구_076.csv
    │   └── ... (지역별 CSV 파일)
    └── 질병데이터 일별전처리
         └──  {year}년_일별_지역별_확진자.csv       # (Output) 전처리 완료 통합 파일
    
```

In [ ]:
import pandas as pd

# ===============================
# 0. 기본 설정 (연도, 지역, 저장파일)
# ===============================
years = [2023, 2024, 2025]

regions = [
    '경기_가평군_075','경기_고양시_덕양구_076','경기_고양시_일산동구_077',
    '경기_고양시_일산서구_078','경기_과천시_079','경기_광명시_080',
    '경기_광주시_081','경기_구리시_082','경기_군포시_083','경기_김포시_084',
    '경기_남양주시_085','경기_동두천시_086','경기_부천시_130',
    '경기_부천시_소사구_087','경기_부천시_오정구_088',
    '경기_성남시_분당구_090','경기_성남시_수정구_091','경기_성남시_중원구_092',
    '경기_수원시_권선구_093','경기_수원시_영통구_094',
    '경기_수원시_장안구_095','경기_수원시_팔달구_096',
    '경기_시흥시_097','경기_안산시_단원구_098','경기_안산시_상록구_099',
    '경기_안성시_100','경기_안양시_동안구_101','경기_안양시_만안구_102',
    '경기_양주시_103','경기_양평군_104','경기_여주시_105',
    '경기_연천군_106','경기_오산시_107',
    '경기_용인시_기흥구_108','경기_용인시_수지구_109','경기_용인시_처인구_110',
    '경기_의왕시_111','경기_의정부시_112','경기_이천시_113',
    '경기_파주시_114','경기_평택시_115','경기_포천시_116',
    '경기_하남시_117','경기_화성시_118','서울_010'
]

date_ranges = {
    2023: {'start_date': '2023-01-01', 'end_date': '2023-12-30'},
    2024: {'start_date': '2023-12-31', 'end_date': '2024-12-28'},
    2025: {'start_date': '2024-12-29', 'end_date': '2025-12-06'}
}

base_output_dir = '/content/drive/MyDrive/AI기상-질병데이터/질병데이터 일별전처리/'

# ===============================
# 1. 연도별 처리
# ===============================
for year in years:
    yearly_records = []

    for region in regions:
        input_path = f"{base_input_dir}{year}년확진자/kdca_{year}_week_{region}.csv"

        try:
            df = pd.read_csv(input_path, encoding='utf-8-sig')

            year_start = pd.to_datetime(f"{year}-01-01")
            first_sunday = year_start + pd.Timedelta(days=(6 - year_start.weekday()) % 7)

            week_columns = [
                col for col in df.columns
                if col.startswith('COLUMN') and col != 'COLUMN1'
            ]

            for _, row in df.iterrows():
                disease = row['SUBTITLE']

                for week_idx, week_col in enumerate(week_columns):
                    weekly_cases = row[week_col]
                    if pd.isna(weekly_cases):
                        weekly_cases = 0

                    daily_cases = weekly_cases / 7
                    week_start = first_sunday + pd.Timedelta(days=week_idx * 7)

                    for d in range(7):
                        yearly_records.append({
                            'date': week_start + pd.Timedelta(days=d),
                            'region': region,
                            'disease_subtitle': disease,
                            'confirmed_cases': daily_cases
                        })

        except FileNotFoundError:
            print(f"❌ 파일 없음: {input_path}")

    df_year = pd.DataFrame(yearly_records)

    dr = date_ranges[year]
    df_year = df_year[
        (df_year['date'] >= dr['start_date']) &
        (df_year['date'] <= dr['end_date'])
    ]

    df_year = df_year.sort_values(
        ['date', 'region', 'disease_subtitle']
    ).reset_index(drop=True)

    output_path = f"{base_output_dir}{year}년_일별_지역별_확진자.csv"

    # 🔴 인코딩 통일 (중요)
    df_year.to_csv(output_path, index=False, encoding='utf-8-sig')

    print(f"✅ 저장 완료: {output_path}")


### 일별·지역별 질병 확진자 데이터와 기상 데이터 매핑 로직 설명

이 코드는 **일별·지역별 질병 확진자 데이터**와

**관측소(ASOS) 기준 일별 기상 데이터**를

**날짜 + 관측 지점 기준으로 병합**하여

**연도별 통합 CSV 파일을 생성**하는 전처리 과정이다.

---

### 1. 전체 처리 목적

이 전처리의 목적은 다음과 같다.

- 질병 데이터
    - 지역 단위
    - 질병별
    - 일별 확진자 수
- 기상 데이터
    - 관측 지점(지점명) 단위
    - 일별 기상 관측값

이를

**“지역 → 대표 기상 관측 지점”으로 매핑한 뒤 날짜 기준으로 병합**하여

머신러닝 학습에 바로 사용할 수 있는 **Feature Dataset**을 생성한다.

---

### 2. 경로 및 디렉터리 설정

### 2.1 입력 데이터 경로

- 질병 데이터
    
    ```
    질병데이터 일별전처리/
    
    ```
    
- 기상 데이터
    
    ```
    2023-2025기상데이터/
    
    ```
    

각 연도별 CSV 파일을 읽어 처리한다.

---

### 2.2 출력 데이터 경로

- 연도별 매핑 결과 저장 위치
    
    ```
    연도별 매핑 데이터/
    
    ```
    

해당 디렉터리가 없을 경우 `os.makedirs`를 통해 자동 생성된다.

---

### 3. 연도별 날짜 범위 정의

연도 경계에서 발생하는 데이터 침범 문제를 방지하기 위해

연도별 유효 날짜 범위를 명시적으로 지정한다.

<표 1> 연도별 날짜 필터 범위

| 연도 | 시작일 | 종료일 |
| --- | --- | --- |
| 2023 | 2023-01-01 | 2023-12-30 |
| 2024 | 2023-12-31 | 2024-12-28 |
| 2025 | 2024-12-29 | 2025-12-06 |

이 범위는 **질병 데이터 기준**으로 적용된다.

---
### 4. 지역 → 지점명 매핑 적용

```python
df_disease['지점명'] = df_disease['region'].map(region_to_station)

```

- 각 지역을 대표 기상 관측 지점으로 변환
- 매핑되지 않는 지역은 `NaN` 처리 후 제거

이 시점부터

**질병 데이터는 행정구역이 아닌 관측 지점 단위 데이터로 해석**된다.

---

### 5 질병–기상 데이터 병합

병합 기준은 다음과 같다.

<표 2> 병합 키

| 질병 데이터 | 기상 데이터 |
| --- | --- |
| date | 일시 |
| 지점명 | 지점명 |
- 병합 방식: `inner join`
- 의미:
    - 질병 데이터와 기상 데이터가 **모두 존재하는 날짜·지점만 유지**

---

### 6. 정렬 및 저장

### 6.1 정렬 기준

병합 후 데이터는 다음 기준으로 정렬된다.

1. date (날짜)
2. 지점명
3. disease_subtitle (질병명)

이는 시계열 모델 학습 시

데이터 순서로 인한 불안정성을 줄이기 위함이다.

---

### 6.2 결과 파일 저장

- 파일명 형식
    
    ```
    {year}년_일별_지역별_확진자_기상매핑.csv
    
    ```
    
- 인코딩: `utf-8-sig`

연도별로 독립적인 학습 데이터셋을 구성할 수 있다.

---

**질병 값은 추정치이며, 기상 값은 실측치**라는 점을 명확히 인지해야 한다.

이 데이터는
- 역학적 인과 분석 ❌
- 머신러닝 기반 예측 모델 학습 ⭕

In [ ]:
import pandas as pd
import os

# ===============================
# 1. 경로 설정
# ===============================
disease_dir = '/content/drive/MyDrive/AI기상-질병데이터/질병데이터 일별전처리/'
weather_dir = '/content/drive/MyDrive/AI기상-질병데이터/2023-2025기상데이터/'
output_dir = '/content/drive/MyDrive/AI기상-질병데이터/년도별 매핑 데이터/'

os.makedirs(output_dir, exist_ok=True)

# ===============================
# 2. 날짜 범위 정의
# ===============================
date_ranges = {
    2023: {'start_date': '2023-01-01', 'end_date': '2023-12-30'},
    2024: {'start_date': '2023-12-31', 'end_date': '2024-12-28'},
    2025: {'start_date': '2024-12-29', 'end_date': '2025-12-06'}
}

# ===============================
# 3. 지역 → 지점명 매핑
# ===============================
mapping_regions = {
    '동두천': [
        '경기_구리시_082','경기_부천시_오정구_088','경기_과천시_079','경기_부천시_130',
        '경기_광명시_080','경기_부천시_소사구_087','경기_고양시_덕양구_076',
        '경기_의정부시_112','경기_고양시_일산동구_077','경기_고양시_일산서구_078',
        '경기_하남시_117','경기_남양주시_085','경기_김포시_084','경기_양주시_103',
        '경기_파주시_114','경기_동두천시_086',
    ],
    '수원': [
        '경기_수원시_팔달구_096','경기_수원시_장안구_095','경기_수원시_권선구_093',
        '경기_화성시_118','경기_수원시_영통구_094','경기_군포시_083',
        '경기_용인시_기흥구_108','경기_의왕시_111','경기_성남시_수정구_091',
        '경기_안산시_단원구_098','경기_성남시_중원구_092','경기_오산시_107',
        '경기_안산시_상록구_099','경기_안양시_동안구_101','경기_용인시_수지구_109',
        '경기_시흥시_097','경기_안양시_만안구_102','경기_성남시_분당구_090',
        '경기_용인시_처인구_110','경기_평택시_115',
    ],
    '양평': ['경기_양평군_104','경기_광주시_081','경기_가평군_075'],
    '이천': ['경기_이천시_113','경기_여주시_105','경기_안성시_100'],
    '파주': ['경기_포천시_116','경기_연천군_106'],
    '서울': ['서울_010'],
}

region_to_station = {
    region: station
    for station, regions in mapping_regions.items()
    for region in regions
}

# ===============================
# 4. 연도별 매핑 처리
# ===============================
for year in [2023, 2024, 2025]:

    print(f"\n===== {year}년 처리 시작 =====")


    # 기상 데이터
    weather_path = f"{weather_dir}OBS_ASOS_DD_{year}.csv"
    df_weather = pd.read_csv(
      weather_path,
      encoding='cp949',
      sep=','
    )

    df_weather['일시'] = pd.to_datetime(df_weather['일시'])

    # 질병 데이터 (연도별 단일 파일)
    disease_path = f"{disease_dir}{year}년_일별_지역별_확진자.csv"
    df_disease = pd.read_csv(
        disease_path,
        encoding='utf-8-sig',
        sep=','
    )
    print("질병컬럼",df_disease.columns)
    print("기상컬럼",df_weather.columns)

    assert df_disease['confirmed_cases'].isna().sum() == 0


    df_disease['date'] = pd.to_datetime(df_disease['date'])



    # 날짜 범위 컷
    dr = date_ranges[year]
    df_disease = df_disease[
        (df_disease['date'] >= dr['start_date']) &
        (df_disease['date'] <= dr['end_date'])
    ]

    # 지역 → 지점명 매핑
    df_disease['지점명'] = df_disease['region'].map(region_to_station)
    df_disease = df_disease.dropna(subset=['지점명'])

    # 병합
    df_merged = pd.merge(
        df_disease,
        df_weather,
        left_on=['date', '지점명'],
        right_on=['일시', '지점명'],
        how='inner'
    )

    # 정렬 (분석 안정성 확보)
    df_merged = df_merged.sort_values(
        by=['date', '지점명', 'disease_subtitle']
    )

    output_path = os.path.join(
        output_dir,
        f"{year}년_일별_지역별_확진자_기상매핑.csv"
    )

    df_merged.to_csv(output_path, index=False, encoding='utf-8-sig')
    print(f"✅ 저장 완료: {output_path}")


###

### 기후 수치형 컬럼 정제 및 최종 Feature Dataset 생성 로직 설명

**질병–기상 매핑이 완료된 일별 데이터**를 입력으로 하여

머신러닝 모델 학습에 사용할 **기후 수치형 Feature Dataset**을 생성하는 전처리 과정이다.

핵심 목적은 다음 세 가지이다.

1. 모델 입력에 필요한 **기후 수치형 컬럼만 선별**
2. 시계열 학습에 방해되는 **결측치(NaN)를 체계적으로 처리**

---

### 1. 입력·출력 경로 설정

### 1.1 입력 데이터

- 경로
    
    ```
    연도별 매핑 데이터/
    
    ```
    
- 파일 형식
    
    ```
    {year}년_일별_지역별_확진자_기상매핑.csv
    
    ```
    

이 데이터는 이미 다음 처리가 완료된 상태이다.

- 질병 데이터 + 기상 데이터 병합
- 날짜 및 지점 기준 정렬
- 기본적인 데이터 정합성 확보

---

### 1.2 출력 데이터

- 경로
    
    ```
    feature_dataset/
    
    ```
    
- 연도별로 독립적인 **최종 Feature Dataset CSV** 생성

---

### 2. 기후 수치형 컬럼 정의

모델 학습에 사용할 **기후 관련 수치형 변수 목록**을 명시적으로 정의한다.

<표 1> 기후 수치형 컬럼 목록

| 구분 | 컬럼 예시 |
| --- | --- |
| 기온 | 평균기온, 최저기온, 최고기온 |
| 습도 | 평균 상대습도, 최소 상대습도 |
| 강수 | 일강수량, 강수 계속시간, 1시간 최다강수량 |
| 바람 | 평균 풍속, 최대 순간 풍속 |
| 일사·일조 | 합계 일조시간, 합계 일사량 |
| 기타 | 지면온도, 이슬점온도, 안개 계속시간 |

이 리스트는 이후 **컬럼 선택·타입 변환·보간의 기준**으로 사용된다.

---

### 3. 출력 컬럼 선별

모델 학습에 필요한 컬럼만 선택한다.

<표 2> 최종 출력 컬럼 구성

| 구분 | 컬럼 |
| --- | --- |
| 키 컬럼 | date, region, disease_subtitle |
| 타깃 | confirmed_cases |
| 기상 식별자 | 지점, 지점명 |
| Feature | climate_numeric_cols |

이 단계 이후 불필요한 문자열·범주형 컬럼은 제거된다.

---

### 3.3 기후 컬럼 타입 정제

```python
pd.to_numeric(errors='coerce')

```

- 문자열, 특수문자 등이 포함된 값은 `NaN`으로 변환
- 이후 보간(interpolation)을 수행하기 위한 **사전 조건 정제 단계**

이 단계가 없으면 보간 로직이 정상 동작하지 않는다.

---

### 4. 이벤트성 기후 변수의 결측 처리

강수·안개와 같은 컬럼은

**값이 없다는 것 자체가 “발생하지 않음”을 의미**하는 경우가 많다.

따라서 아래 컬럼은 보간이 아닌 **0으로 직접 대치**한다.


 * 이벤트성 컬럼 0 대치 대상(컬럼)

- 일강수량(mm)

- 강수 계속시간(hr)

- 1시간 최다강수량(mm)

- 안개 계속시간(hr)

이 처리는 **기후 데이터의 의미적 왜곡을 최소화하기 위한 전략**이다.

---

### 5. 날짜 정규화 및 정렬

### 5.1 날짜 정규화

```python
df_output['date'].dt.normalize()

```

- 시간 정보를 제거하고 날짜 단위로 통일
- 시계열 모델 학습 시 불필요한 시간 편차 제거

---

### 5.2 정렬 기준

다음 기준으로 **안정 정렬(mergesort)**을 수행한다.

1. region
2. disease_subtitle
3. date

이는 이후 `groupby + transform` 기반 보간 시

**시계열 순서가 깨지지 않도록 보장**한다.

---

### 6. 결측치 보간 전략

이 코드의 핵심은 **단계적 결측치 처리 전략**이다.

---

### 6.1 1차 결측 보간: 선형 보간

<코드 1> 1차 보간 개념

- 그룹 기준
    
    ```
    region + disease_subtitle
    
    ```
    
- 동일 지역·질병 내에서만 시계열 보간 수행
- 방법: `linear`
- 방향: 앞·뒤 모두 허용 (`limit_direction='both'`)

즉,

**서로 다른 지역이나 질병의 값이 섞이지 않는다.**

---

### 6.2 보간 후 결측치 점검

보간 이후에도 남아 있는 결측치를 출력하여

데이터 상태를 명시적으로 확인한다.

이 단계는 **전처리 결과를 검증하기 위한 안전장치**이다.

---

### 6.3 2차 결측 처리: 컬럼별 개별 전략

1차 보간 후에도 남은 결측치는 **컬럼 특성에 따라 개별 처리**한다.

- 안개 계속시간(hr), 강수 계속시간(hr)
    
    → 0 대치
    
- 합계 일사량(MJ/m2)
    
    → 1차 보간 실패 시 0 대치
    
    관측 누락 가능성 고려
      합계 일사량(MJ/m2) 값이 결측치가 많은 이유는  6개의 관측소중 2개의 관측소에서 합계일사량 데이터가 자체가 측정이 안되는 오류가 있어 보였다 그리하여 값이 없으면 0 으로 결측치를 처리하였다
    

이로써 **기후 수치형 컬럼의 NaN을 완전히 제거**한다.

---

### 7. 최종 결과 저장

- 파일명 형식
    
    ```
    {year}년도_기후_컬럼정의(최종보간_1227).csv
    
    ```
    
- 인코딩: `utf-8-sig`
- 모든 기후 수치형 컬럼에 대해 NaN = 0 보장

이 데이터는 **추가 전처리 없이 바로 모델 학습에 사용 가능**하다.

---

### 8. 결과 데이터의 성격과 한계

이 Feature Dataset의 성격은 다음과 같다.

- 질병 데이터
    - 주간 데이터를 일별로 균등 분배한 값 (추정치)
- 기후 데이터
    - 실측 관측값 + 시계열 보간값 혼합

In [ ]:


import pandas as pd
import os

# ===============================
# 0. 경로 설정
# ===============================
input_dir = '/content/drive/MyDrive/AI기상-질병데이터/년도별 매핑 데이터/'
output_dir = '/content/drive/MyDrive/AI기상-질병데이터/feature_dataset/'

os.makedirs(output_dir, exist_ok=True)

years = [2023, 2024, 2025]

# ===============================
# 1. 기후 수치형 컬럼
# ===============================
climate_numeric_cols = [
    '평균기온(°C)','최저기온(°C)','최고기온(°C)',
    '평균 이슬점온도(°C)','평균 상대습도(%)','최소 상대습도(%)',
    '평균 증기압(hPa)','일강수량(mm)','강수 계속시간(hr)',
    '1시간 최다강수량(mm)','평균 풍속(m/s)','최대 순간 풍속(m/s)',
    '합계 일조시간(hr)','합계 일사량(MJ/m2)',
    '평균 지면온도(°C)','안개 계속시간(hr)'
]

# ===============================
# 2. 연도별 처리
# ===============================
for year in years:
    print(f"\n===== {year}년 기후 컬럼 데이터 생성 시작 =====")

    input_path = f"{input_dir}{year}년_일별_지역별_확진자_기상매핑.csv"

    df = pd.read_csv(input_path, encoding='utf-8-sig')
    df['date'] = pd.to_datetime(df['date'])

    # -------------------------------
    # 출력 컬럼 선택
    # -------------------------------
    output_cols = ['date', 'region', 'disease_subtitle','confirmed_cases' , '지점', '지점명'] + climate_numeric_cols

    df_output = df[output_cols].copy()

     # -------------------------------
    # 타입 정제 (보간 전 필수)
    # -------------------------------
    for col in climate_numeric_cols:
        df_output[col] = pd.to_numeric(df_output[col], errors='coerce')

    # 결측치 보정(

    rain_cols = [
        '일강수량(mm)',
        '강수 계속시간(hr)',
        '1시간 최다강수량(mm)',
        '안개 계속시간(hr)'  # 안개도 이벤트형이면 포함
    ]

    df_output[rain_cols] = df_output[rain_cols].fillna(0)


    # -------------------------------
    # 날짜 정규화 (시계열 안정성)
    # -------------------------------
    df_output['date'] = pd.to_datetime(df_output['date']).dt.normalize()

    # -------------------------------
    # 정렬 (AI 시계열 학습 필수)
    # -------------------------------
    df_output = df_output.sort_values(
        by=['region', 'disease_subtitle', 'date'],
        ascending=[True, True, True],
        kind='mergesort'   # 안정 정렬
    ).reset_index(drop=True)

    # -------------------------------
    # 🔥 결측 보간 (1차: 선형 보간)
    # -------------------------------
    df_output[climate_numeric_cols] = (
        df_output
        .groupby(['region', 'disease_subtitle'])[climate_numeric_cols]
        .transform(lambda x: x.interpolate(method='linear', limit_direction='both'))
    )

    print('\n확인: 보간 후 남은 결측치 (')
    print(df_output[climate_numeric_cols].isnull().sum().sort_values(ascending=False))
    # -------------------------------
    # 📍 추가 결측치 처리 전략 적용 (2차: 특정 컬럼)
    # -------------------------------
    # 1. 안개 계속시간(hr), 강수 계속시간(hr): 0 대치
    df_output['안개 계속시간(hr)'] = df_output['안개 계속시간(hr)'].fillna(0)
    df_output['강수 계속시간(hr)'] = df_output['강수 계속시간(hr)'].fillna(0)

    # 2. 합계 일사량(MJ/m2): 1차 보간 후에도 남은 결측치는 0으로 대치
    df_output['합계 일사량(MJ/m2)'] = df_output['합계 일사량(MJ/m2)'].fillna(0)


    # -------------------------------
    # 저장
    # -------------------------------
    output_path = f"{output_dir}{year}년도_기후_컬럼정의(최종보간_1227).csv" # Changed output filename

    df_output.to_csv(
        output_path,
        index=False,
        encoding='utf-8-sig'
    )

    print(f"✅ {year}년도 기후 컬럼정의 CSV 저장 완료")
    print(df_output.head(2))
    # 최종적으로 모든 NaN이 제거되었는지 확인
    print(f"Remaining NaNs after final imputation for {year} in climate numeric columns:")
    print(df_output[climate_numeric_cols].isnull().sum().sum())



===== 2023년 기후 컬럼 데이터 생성 시작 =====

확인: 보간 후 남은 결측치 (
합계 일사량(MJ/m2)    28392
평균기온(°C)             0
최고기온(°C)             0
최저기온(°C)             0
평균 상대습도(%)           0
최소 상대습도(%)           0
평균 증기압(hPa)          0
평균 이슬점온도(°C)         0
일강수량(mm)             0
강수 계속시간(hr)          0
평균 풍속(m/s)           0
1시간 최다강수량(mm)        0
최대 순간 풍속(m/s)        0
합계 일조시간(hr)          0
평균 지면온도(°C)          0
안개 계속시간(hr)          0
dtype: int64
✅ 2023년도 기후 컬럼정의 CSV 저장 완료
        date      region disease_subtitle  confirmed_cases   지점 지점명  평균기온(°C)  \
0 2023-01-01  경기_가평군_075              뎅기열              0.0  202  양평      -1.0   
1 2023-01-02  경기_가평군_075              뎅기열              0.0  202  양평      -4.2   

   최저기온(°C)  최고기온(°C)  평균 이슬점온도(°C)  ...  평균 증기압(hPa)  일강수량(mm)  강수 계속시간(hr)  \
0      -6.2       4.9         -10.5  ...          2.9       0.0          0.0   
1      -8.4       0.6         -14.9  ...          1.9       0.0          0.0   

   1시간 최다강수량(mm)  평균 풍속(m/s)  최대 순간 풍속(m/s)  합계 일조시간(